In [ ]:
!pip install selenium

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install pyMuPDF

In [ ]:
pip install tabula-py

In [ ]:
!pip install pdfminer

In [ ]:
import re
import io

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#import requests library
import requests
#the website URL
url_link = "http://www.zionsville-in.gov/Archive.aspx?AMID=68&Type=&ADID="
result = requests.get(url_link).text
doc = BeautifulSoup(result, "html.parser")

print(doc.prettify())


In [ ]:
def info(pdf_path):
 
    # used get method to get the pdf file
    response = requests.get(pdf_path)
 
    # response.content generate binary code for
    # string function
    with io.BytesIO(response.content) as f:
 
        # initialized the pdf
        pdf = PdfFileReader(f)
 
        # all info about pdf
        information = pdf.getDocumentInfo()
 
    txt = f"""
    Information about {pdf_path}:
     
    Author: {information.author}
    Creator: {information.creator}
    Producer: {information.producer}
    Subject: {information.subject}
    Title: {information.title}
    """
    print(txt)
     
    return information

In [ ]:
# created an empty list for putting the pdfs
list_of_pdf = set()
 
# accessed the first p tag in the html
l = doc.find('p')
 
# accessed all the anchors tag from given p tag
p = l.find_all('a')
 
# iterate through p for getting all the href links
for link in p:
     
    # original html links
    print("links: ", link.get('href'))
     
    # converting the extension from .html to .pdf
    pdf_link = (link.get('href')[:-5]) + ".pdf"
     
    # converted to .pdf
    print("converted pdf links: ", 'http:/'+pdf_link)
    print("\n")
     
    # added all the pdf links to set
    list_of_pdf.add(pdf_link)

In [ ]:
for i in list_of_pdf:
    info(i)

In [ ]:
import tabula

pdf_path = "/content/Resolution 2023-06 - Regarding the Appropriation of Additional Funds (Police Vehicles).pdf"

dfs = tabula.read_pdf(pdf_path, pages = '2')
print(dfs)

[]


In [ ]:
!pip install PIL
!pip install pytesseract
!pip install pdf2image


In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
# import libs
try:
    from PIL import Image
except ImportError:
import Image
import cv2
import pytesseract
import os
import numpy as np
from pdf2image import convert_from_bytes

# Some help functions 
def get_conf(page_gray):
    '''return a average confidence value of OCR result '''
    df = pytesseract.image_to_data(page_gray,output_type='data.frame')
    df.drop(df[df.conf==-1].index.values,inplace=True)
    df.reset_index()
    return df.conf.mean()
  
def deskew(image):
    '''deskew the image'''
    gray = cv2.bitwise_not(image)
    temp_arr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(temp_arr > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated
  
'''
Main part of OCR:
pages_df: save eextracted text for each pdf file, index by page
OCR_dic : dict for saving df of each pdf, filename is the key
'''
 
OCR_dic={} 
for file in list_of_pdf:
    # convert pdf into image
    pdf_file = convert_from_bytes(open(os.path.join(PATH,file), 'rb').read())
    # create a df to save each pdf's text
    pages_df = pd.DataFrame(columns=['conf','text'])
    for (i,page) in enumerate(pdf_file) :
        try:
            # transfer image of pdf_file into array
            page_arr = np.asarray(page)
            # transfer into grayscale
            page_arr_gray = cv2.cvtColor(page_arr,cv2.COLOR_BGR2GRAY)
            # deskew the page
            page_deskew = deskew(page_arr_gray)
            # cal confidence value
            page_conf = get_conf(page_deskew)
            # extract string 
            pages_df = pages_df.append({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)}, ignore_index=True)
        except:
            # if can't extract then give some notes into df
            pages_df = pages_df.append({'conf': -1,'text': 'N/A'}, ignore_index=True)
            continue
    # save df into a dict with filename as key        
    OCR_dic[file]=pages_df
    print('{} is done'.format(file))